In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [2]:
class LogisticRegression:
    def __init__(self, lr = 0.1, eps = 1e-6, num_iter = 1000, verbose = False):
        self.lr = lr
        self.eps = eps
        self.num_iter = num_iter
        self.verbose = verbose
        self.loss_dis = []

    def add_intercept(self, X):
        intercept = np.ones((X.shape[0],1))
        return np.concatenate((intercept, X))
    
    @staticmethod
    def stable_sigmoid(x):
       z = np.zeros_like(x)
       z[x >= 0] = 1/(1 + np.exp(-x))
       z[x < 0] = np.exp(x[x < 0]) 
       z[x < 0] = z[x < 0] / (1 + z[ x < 0])
       return z
    
    def loss(self, m, y, y_hat):
        return -(1/m)*(y.T @ np.log(y_hat + self.eps) + (1-y).T @ np.log(1-y_hat + self.eps))
    
    def grad_desc(self, X, y):
        m = len(y)
        y_hat = self.stable_sigmoid( np.dot(X, self.theta))
        grad = (1/m)* X.T @ (y_hat.T - y.T)
        new_theta = self.theta - self.lr*grad
        return new_theta
    
    def fit(self, X, y):
        samples, features = X.shape[:2]

        X = self.add_intercept(X)

        self.theta = np.zeros(features+1)

        pbar = tqdm(range(self.num_iter))

        for epoch in pbar:

            y_hat = self.stable_sigmoid( X @ self.theta)

            new_theta = self.grad_desc(X,y) 

            if np.linalg.norm(new_theta - self.theta, 1) < self.eps:
                self.theta = new_theta
                break

            self.theta = new_theta

            if self.verbose is True:
                loss = self.loss(samples, y, y_hat)
                self.loss_dis.append(loss)
                pbar.set_description(f'epoch {epoch}: loss = {loss:.2f}')   

    def predict(self, X):
        y_pred = self.stable_sigmoid( X @ self.theta)
        return (y_pred >= 0.5).astype(int)

    def display_loss(self):
        plt.figure(figsize = (10,7))
        plt.plot(self.loss_dis)
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.show()

    def accuracy(self, X, y):
        y_pred = self.predict(X)
        accuracy = np.mean(y_pred == y)
        print(f'Accuracy: {accuracy:.3}')